# 3.3. 将课程内容转为PPT

## 🚄 前言
在实际授课过程中，常常需要制作PPT以辅助讲解。本节课程介绍如何借助文本生成模型 Qwen-Max 和基于 Markdown 的演示文稿生成工具 Marp，将课程内容快速转换为 PPT。

## 🍁 课程目标
学完本课程后，你将能够：
- 使用 Qwen-Max 进行内容提炼和格式转换
- 了解 Marp，并使用 Marp 将 Mardkown 文档导出为 PPT

## 📖 课程目录

- [1. 原理介绍](#🧮-1-原理介绍)
- [2. 代码实践](#🛠️-2-代码实践)
    - [2.1. 环境准备](#21-环境准备)
    - [2.2. 设置 API 客户端](#22-设置-api-客户端)
    - [2.3. 从课程内容中提炼出 PPT 脚本](#23-从课程内容中提炼出-ppt-脚本)
    - [2.4. 将 PPT 脚本转换为 Marp 格式](#24-将-ppt-脚本转换为-marp-格式)
    - [2.5. 导出为 PPT](#25-导出为-ppt)
  

## 🧮 1. 原理介绍

为了将课程内容转换成 PPT，除了大模型，我们还需要结合使用一款工具：

- [Marp](https://github.com/marp-team/marp)： 一个基于 Markdown 的幻灯片生成工具，旨在简化创建演示文稿的过程。它允许用户使用轻量级的 Markdown 语法编写幻灯片，生成美观的演示，无需使用复杂的幻灯片编辑器。这使得 Marp 特别适合技术人员、开发者和喜欢使用 Markdown 的用户。


结合使用 Qwen-Max 和 Marp 将课程内容转为 PPT 的过程如下：

<div align="center">
    <img src="https://gw.alicdn.com/imgextra/i1/O1CN01ywrWDR1dI9CB5NFFR_!!6000000003712-0-tps-2398-190.jpg" alt="流程" width="80%"/>
</div>


## 🛠️ 2. 代码实践

接下来，让我们运行以下代码，将第一节课生成的课程内容转换为 PPT。

### 2.1. 环境准备

1. 安装 Python 库工具。

In [1]:
! pip install -r requirements.txt -q

2. 你需要安装一些依赖项以完成后面的工作。

首先请安装 marp cli 库，这一步可能会**需要一些时间**，一般十分钟左右会完成，如果卡在某个进度，请尝试（🛑）终止进程重新执行。

如果这里的 marp cli 安装不成功，请检查网络链接，尝试切换到命令行状态安装。

运行以下代码来安装依赖项。

In [ ]:
# 安装marp cli
! npm install -g @marp-team/marp-cli -y -qq

3. 导入必要的模块。

In [1]:
import os
import openai
import json
from typing import Union
import subprocess
from utils import read_text_from_file, save_file,load_config

4. 设置环境变量

In [7]:
import os
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

5. 加载配置文件。

In [3]:
project_config = load_config("config.json")
title = project_config["title"]

### 2.2. 设置 API 客户端
设置 OpenAI 的 API 客户端，用于后续调用阿里云百炼的 Qwen-Max 模型和 Flux-Merged 模型。

In [4]:
client = openai.OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

### 2.3. 从课程内容中提炼出 PPT 脚本

首先，我们让文本生成模型 Qwen-Turbo 扮演一位 PPT 专家，从详细的课程内容中提炼出用于演示的内容要点。

1. 定义一个 `create_ppt_script` 函数，用于使用 Qwen-Turbo 将课程内容从源语言直接翻译为目标语言。

In [5]:
def create_ppt_script(course_script):
    system_message = "你是一位专业的 PPT 专家，专注于根据课程脚本开发教学 PPT。"

    prompt = (
        f"将以下课程脚本转换为 PPT 格式：\n\n"
        f"{course_script}\n\n"
        "请遵循以下指导原则：\n"
        "保持原有的章节结构，注意标题页和总结页，此外中间的页面不要额外生成内容。"
        "仅生成PPT框架的内容。结尾不要写任何寄语，会被当作一个新的片段。在ppt里不要写任何备注。"
        "1. **提炼关键信息**：避免冗长段落。\n"
        "2. **使用简短标题**：聚焦于要点。\n"
        "3. **内容简洁明了**：去除多余信息。\n"
        "4. **结合图片辅助**：在适当的位置插入图片。\n"
        "5. **不新增内容**：确保不添加任何额外信息。\n"

        "格式要求：\n"
        "1. 正文使用清晰的有序和无序列表：有序列表用于顺序内容，无序列表用于其它内容。\n"
        "2. 确保代码片段格式规范。\n"
        "3. 输出必须是 Markdown 格式。\n\n"
        "\n 不要输出 ### 三级标题\n"
        "\n 遵照原文不要生成新的 ## 二级标题\n"
        "4. 允许使用 1 级和 2 级标题，禁止出现 3 级及以上标题。\n"
        "5. 输出中不得包含 ```markdown 和 ```。\n"
        "6. 图片不显示为列表。\n"

        "输出要求：\n"
        "输出应仅包含 Markdown 格式的 PPT 脚本，禁止任何解释性文字。\n"
        "示例输出格式：\n"
        "# 云计算深入探索\n"
        "![云计算深入探索](https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/2c/20240923/f1ceb422/56a52df0-426f-4352-bbf6-40ffeb1cb04e-1.png?Expires=1727165843&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=YaCArgLeSs4y3SLcROEBfiUnACc%3D)\n"
        "## 什么是云计算？\n"
        "- 通过互联网提供服务\n"
        "- 按需获取\n"
        "- 按量付费\n"
        "![什么是云计算](https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/aa/20240923/f1ceb422/8681083c-c304-45fd-af78-ecb68174561d-1.png?Expires=1727165862&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=I3MMGZA3mcQWZ24yz7qkdPl5G2Y%3D)\n"
    )

    # 直接调用 API 获取课程脚本
    completion = client.chat.completions.create(
        model="qwen-max",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    # 如果 model_dump_json() 返回字符串需要解析为 JSON
    dumped_json = json.loads(completion.model_dump_json())

    # 使用正确的属性访问生成的内容
    return dumped_json['choices'][0]['message']['content']

2. 调用 `create_ppt_script` 函数，从课程脚本中提炼出 PPT 脚本。

In [6]:
# 从指定目录读取首个文件内容
course_script_file = project_config["course_script_with_illustrations_file_path"].format(title=project_config["title"])

# 读取课程脚本
course_script = read_text_from_file(course_script_file)

# 确保课程脚本不为空
if not course_script:
    raise ValueError("课程脚本为空，无法生成 PPT。")

# 调用函数生成 PPT 脚本
ppt_script = create_ppt_script(course_script)
print(ppt_script)

# 构造文件名
ppt_script_file_path = project_config["ppt_script_file_path"].format(title=project_config["title"])

# 调用函数保存PPT
save_file(ppt_script, ppt_script_file_path)


成功读取文件：./output/course_script_with_illustrations/云计算_含插画课程脚本.md
```markdown
# 云计算基础课程
## 什么是云计算？
- 计算资源互联网提供
- 需求驱动，即时扩展
- 按需付费，成本效率

![云计算比喻](../illustrations/云计算比喻.png)

## 云计算核心优势
### 灵活性与可扩展性
- 快速调整服务规模

![灵活变形车](../illustrations/灵活变形车.png)

### 成本效益
- 减少硬件投资

![租房vs买房](../illustrations/租房vs买房.png)

### 易于管理
- 服务商管理基础架构

![超级管家](../illustrations/超级管家.png)

## 服务模型
### IaaS (基础设施即服务)
- 计算、存储、网络资源

![IaaS厨房](../illustrations/IaaS厨房.png)

### PaaS (平台即服务)
- 包含开发工具与数据库

![PaaS半成品](../illustrations/PaaS半成品.png)

### SaaS (软件即服务)
- 完整应用，浏览器访问

![SaaS外卖](../illustrations/SaaS外卖.png)

## 云部署模式
### 公有云
- 第三方提供，公众访问

![公有云商场](../illustrations/公有云商场.png)

### 私有云
- 专为企业，内部使用

![私有云会所](../illustrations/私有云会所.png)

### 混合云
- 结合公有与私有云优势

![混合云冰箱超市](../illustrations/混合云冰箱超市.png)
```
请注意，上述Markdown格式的PPT脚本已根据您的要求进行调整，包含了1级和2级标题，没有新增任何内容，并且整合了图片引用。
目标目录：./output/ppt_script
文件路径：./output/ppt_script/云计算_幻灯片脚本.md
文件已成功保存为：./output/ppt_script/云计算_幻灯片脚本.md


### 2.4. 将 PPT 脚本转换为 Marp 格式

接下来，我们使用 Qwen-Max，将 Markdown 格式的 PPT 脚本转换为 Marp 格式。

1. 定义一个 `convert_md_to_marp` 函数，用于转换 PPT 脚本为 Marp 格式。

In [32]:
def convert_md_to_marp(ppt_script):
    system_message = "你是一位专业的 Marp 专家，专注于将内容从 Markdwon 转换为 Marp 格式。"

    prompt = (
        f"请将以下内容转换为 Marp 格式：\n\n"
        f"{ppt_script}\n\n"
        "请遵循以下要求：\n"
        "1. **Marp 固定开头格式**：\n"
        "   ---\n"
        "   marp: true\n"
        "   theme: default\n"
        "   paginate: true\n"
        "   size: 16:9\n"
        "   ---\n"
        "2. **封面和封底图片**格式为：`![bg](url)`。\n"
        "3. **正文图片**格式为：`![bg left](url)` 或 `![bg right](url)`。\n"
        "4. **结尾**不得包含 `---`。\n"
        "5. **一级标题**和**二级标题**之间必须使用 `---` 分隔。\n"
        "6. 生成的内容开头和结尾不包含 ```。\n"
        "\n输出要求：\n"
        "\n不要输出 ### 三级标题\n"
        "\n遵照原文不要生成新的 ## 二级标题\n"
        "输出应仅包含 Marp 格式的 PPT 脚本，不要添加其他内容。\n"
        "示例输出格式：\n"
        "---\n"
        "marp: true\n"
        "theme: default\n"
        "paginate: true\n"
        "size: 16:9\n"
        "---\n"
        "![bg](https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/2c/20240923/f1ceb422/56a52df0-426f-4352-bbf6-40ffeb1cb04e-1.png?Expires=1727165843&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=YaCArgLeSs4y3SLcROEBfiUnACc%3D)\n"
        "# 云计算深入探索\n"
        "---\n"
        "![bg left](https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/aa/20240923/f1ceb422/8681083c-c304-45fd-af78-ecb68174561d-1.png?Expires=1727165862&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=I3MMGZA3mcQWZ24yz7qkdPl5G2Y%3D)\n"
        "## 什么是云计算？\n"
        "- 通过互联网提供服务\n"
        "- 按需获取\n"
        "- 按量付费\n"
    )
    # 直接调用 API 获取课程脚本
    completion = client.chat.completions.create(
        model="qwen-max",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    # 如果 model_dump_json() 返回字符串需要解析为 JSON
    dumped_json = json.loads(completion.model_dump_json())

    # 使用正确的属性访问生成的内容
    text = dumped_json['choices'][0]['message']['content']
    text = text.replace("```\n","").replace("\n```","")
    return text

2. 调用 `convert_md_to_marp` 函数转换 PPT 脚本为 Marp 格式。

In [3]:
ppt_script_file_path = project_config["ppt_script_file_path"].format(title=project_config["title"])
ppt_script = read_text_from_file(ppt_script_file_path)

# 确保 PPT 脚本不为空
if not ppt_script:
    raise ValueError("PPT 脚本为空，无法进行转换。")

# 调用函数生成 Marp 格式的脚本
marp_script = convert_md_to_marp(ppt_script)
print(marp_script)

# 构造文件名
marp_script_file_path = project_config["marp_script_file_path"].format(title=project_config["title"])

# 保存marp文件
save_file(marp_script, marp_script_file_path)


### 2.5. 导出为 PPT

#### 2.5.1. 安装依赖项

**安装浏览器**

你需要安装一款浏览器来完成后面的工作
- 如果你在使用 MAC上的IDE如code，你可以确认自己的机器上是否安装了 chrome 或 firefox。如果已经安装浏览器，你可以**跳过这一步**。
- 如果你在使用 pai-DSW 或者 modelscope-DSW，我们推荐你执行以下代码，安装linxu版本的 firefox。 

你可以取消下面代码的注释来 [【安装 firefox】](https://support.mozilla.org/en-US/kb/install-firefox-linux) 以完成后面的工作。请注意，经过这一步，你应该安装了一个超过200MB的firefox。

这个安装需要等待时间较长，根据网速不同可能需要几分钟到几小时。等待安装完成后就可以继续做PPT转换。

> 如果你在DSW上仍然频繁遇到各种环境配置问题，可以考虑将课程代码下载到本地电脑上运行。这样，你将拥有更高的权限，并且更容易搭建所需的环境。（请参考第一章计算环境准备）

In [ ]:
# # 如果你本机上没有安装 chrome 或 firefox，你需要取消下面代码行的注释，去掉“# ”，执行该语句可以安装浏览器firefox。

# ! /bin/bash install_firefox.sh -qq

**安装其他依赖项**

In [ ]:
! sudo apt-get update -qq
# 安装依赖
! apt-get install -y poppler-utils -qq

# 安装python依赖
! pip install pdf2image -qq

#### 2.5.2. 生成PPT 

最后，我们可以使用 Marp 将生成的 Marp 格式 PPT 导出为 PPTX，用于授课时演示

1. 定义一个 `export_marp_ppt` 函数，用于导出 PPT。

函数内容我们保存在convert_md_pdf.py中，可以从命令行调用。

2. 调用 `export_marp_ppt` 函数导出 PPT。

In [9]:
marp_script_file_path = project_config["marp_script_file_path"].format(title=project_config["title"])
marp_script = read_text_from_file(marp_script_file_path)

# 确保 Marp 脚本不为空
if not marp_script:
    raise ValueError("Marp 脚本为空，无法生成 PPT。")


marp_export_folder = project_config["marp_export_folder"]

! python convert_md_pdf.py {marp_script_file_path}  {marp_export_folder}

#### QA
如果你在这里出现了找不到firefox浏览器的报错，你可以先请检查 2.5.1 中  ```! /bin/bash install_firefox.sh```这句是不是没有执行。去掉“# ”可以取消注释，之后您可以执行这句安装。

## ✅ 本节小结

- 在本次学习和实践中，我们使用了 Qwen-Max 进行了内容提炼和格式转换，并且使用了 Marp 将 Markdown 文件导出为 PPT。
- 完成授课后，不少同学希望能有课程录音，便于之后随时随地复习巩固。接下来，我们将学习如何使用大模型将课程内容转换为音频。

## 🔥 课后小测验

【单选题】3.3.1. 你想使用 Qwen-Max 将一段冗长的课程文本提炼成简洁的 PPT 脚本要点。以下哪种 Prompt 更有效？（ ）

A. "请阅读以下文本：[文本内容] "

B. "请将以下文本总结成 PPT 脚本要点，每页 PPT 对应一个要点：[文本内容]"

C. "请生成一张 PPT"

D. "使用 Marp 生成 PPT"

答案：B 选项 B 清晰地指示了 Qwen-Max 的任务和期望的输出格式，比其他选项更有效。选项 A 只是让模型阅读文本，没有明确的任务。选项 C 和 D 与提炼文本无关。

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。